### Make predition of stars for Training dataset based on our models (text mining model and random forest model):
#### 1. Perform 5_fold cross_validation to get predition for every entry of our training dataset 
#### 2. Repeat the previous step for 20 times and take mean as our predition 

### Combine those two prediction and use a Gradient Boosting mode to predict the stars of each business


In [74]:
import numpy as np
import time
import imp
import sklearn as sk
import random
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import json
import csv
import unicodecsv
import codecs
from collections import defaultdict
import sys
import lda
import csv
#imp.reload(sys)
#sys.stdout = codecs.getwriter('utf-8')(sys.stdout)
#sys.stderr = codecs.getwriter('utf-8')(sys.stderr)
import pickle
from collections import Counter
import itertools
import nltk
import re
import math
from nltk.probability import *
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
review_list = []
with open('/Users/Ellie/Documents/BERKELEY/STAT154/Proj/data_updata/yelp_academic_dataset_review_train.csv', encoding = 'utf8') as f:
    reader = csv.reader(f, delimiter=',')
    for line in reader:
        review_list.append(tuple(line))
        
review_list.pop(0)

('',
 'funny',
 'user_id',
 'review_id',
 'text',
 'business_id',
 'stars',
 'date',
 'useful',
 'type',
 'cool')

In [91]:
business_review = pd.read_csv('/Users/Ellie/Documents/BERKELEY/STAT154/Proj/data_updata/outputchin.csv', index_col=0)
text = pd.read_csv("~/Documents/BERKELEY/STAT154/Proj/out5.csv")


In [58]:
business_id = [element[5] for element in review_list]
all_business_id = np.unique(business_id)
stars = [int(element[6]) for element in review_list]
real_value = pd.DataFrame({'business_id': business_id, 'stars': stars})
real_value = real_value.groupby('business_id')['stars'].mean()

In [4]:
# perform cross validation
cv_id = []
for i in range(0, 5):
    cv_id.extend([i] * 502)

In [5]:
# Predict stars for training dataset based on text mining model
sample = [[] for i in range(len(review_list))]
for j in range(0, 20): 
    random.shuffle(cv_id)
    for i in range(0,5):
        train_id = [all_business_id[j] for j in range(0, len(all_business_id)) if cv_id[j] != i]
        test_id = [all_business_id[j] for j in range(0, len(all_business_id)) if cv_id[j] == i]
        #         train_id = [all_business_id[i - 1] for i in train_business]
        #         test_id = [all_business_id[i - 1] for i in test_buisness]
        train_index = [i for i in range(0, len(review_list)) if review_list[i][5] in train_id]
        test_index = [i for i in range(0, len(review_list)) if review_list[i][5] in test_id]

        train_list = [review_list[j][4] for j in train_index]
        train_response = [review_list[j][6] for j in train_index]
        #     train_list_id = [element[5] for element in train_set]
        test_list = [review_list[j][4] for j in test_index]
        test_response = [review_list[j][6] for j in test_index]
        #     test_list_id = [review_list[j][5] for j in test_index]

        text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),('clf', SGDClassifier(loss='hinge', penalty='l2', alpha = 3.4322448979591842e-05, n_iter=5, random_state=42))])
        text_clf = text_clf.fit(train_list, train_response)
        pred = text_clf.predict(test_list)
        pred = pred.astype(int).tolist()
        for i in test_index:
            sample[i].append(pred.pop(0))
#     result = pd.DataFrame({'business_id': test_list_id, 'stars': pred})
#     result_agg = result.groupby(by = ['business_id'])['stars'].mean()
#     result_agg = result_agg.reset_index()
    
#     result_act = pd.DataFrame({'business_id': test_list_id, 'stars': list(map(int, test_response))})
#     result_act = result_act.groupby(by = ['business_id'])['stars'].mean()
#     result_act = result_act.reset_index()    
#     combine = pd.merge(result_agg, result_act, on = 'business_id')

In [9]:
train_list_id = [element[5] for element in review_list]

In [6]:
sample_mean = [float(np.mean(i)) for i in sample]

In [14]:
## text is a dataframe of predicted stars 
text = pd.DataFrame({'business_id': train_list_id, 'stars': sample_mean})
text = text.groupby(by = ['business_id'])['stars'].mean()
text = text.reset_index()


2510

In [34]:
# perform cross validation
cv_id = []
for i in range(0, 5):
    cv_id.extend([i] * 495)
cv_id.extend([0,0])

116260

In [36]:
# Predict stars for training dataset based on random forest model
business = [[] for i in range(len(review_list))]
for j in range(0, 20):
    random.shuffle(cv_id)
    for i in range(0, 5):
        train_index = [index for index in range(0, len(business_review)) if cv_id[index] != i]
        test_index = [index for index in range(0, len(business_review)) if cv_id[index] == i]
        train_business = business_review.iloc[train_index]
        test_business = business_review.iloc[test_index]
        train = train_business.drop('stars', axis=1)
        train = train.drop('business_id', axis=1)
        test = test_business.drop('stars', axis=1)
        test = test.drop('business_id', axis=1)
        train_y = train_business['stars']
        clr = RandomForestRegressor(n_jobs = 5, n_estimators = 1000)
        clr.fit(train,train_y)
        predictions = clr.predict(test)
        predictions = predictions.astype(float).tolist()
        for t in test_index:
            business[t].append(predictions.pop(0))


In [52]:
business1 = business[0:len(business_review)]
sample_business = [float(np.mean(i)) for i in business1]
bus = pd.DataFrame({'business_id': business_review.business_id, 'stars': sample_business})

In [62]:
real_value = real_value.reset_index()

In [64]:
final = pd.merge(text, bus, on = 'business_id')
final = pd.merge(final, real_value, on = 'business_id')

In [72]:
final_train = final[[1,2]].as_matrix()
final_response = final[[3]].as_matrix()
final_response = [float(final_response[i]) for i in range(0, len(final_response))]            

In [93]:
final_testing = test_business[[1,2]]


In [94]:
# Gradient Boosting Model
boost_lm = GradientBoostingRegressor()
boost_lm.fit(final_train, final_response)
pred = boost_lm.predict(final_testing)
result = pd.DataFrame({'business_id': test_business.business_id, 'stars': pred})
result.to_csv("out5.csv")